In [ ]:
lvl           = None
file_path_Sgm = None
file_path_Lvl = None

In [ ]:
%run ./5___Origins___Functions.ipynb

---
---
---

### Get all 26's and all possible pair elements

Since we do this by shifting the cube, we automatically check for any connections over the edges.

In [ ]:
with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)
size = grid_lvld.shape[0]
s_r  = range(size)

In [ ]:
grid_26 = np.full(grid_lvld.shape, True); grid_pairs = np.full(grid_lvld.shape, True)

for offs in offsets:
    grid_26    &= ((grid_lvld - shift_cube(grid_lvld, offs)) <  0)
    grid_pairs &= ((grid_lvld - shift_cube(grid_lvld, offs)) <= 0)

grid_pairs &= ~grid_26

### Make a list of 26's (isolated origins)

In [ ]:
dict_origins_isolated = {val: [] for val in range(0, lvl)}

for isolated_i in np.argwhere(grid_26):
    dict_origins_isolated[grid_lvld[tuple(isolated_i)]].append(isolated_i.tolist())

In [ ]:
with open(file_path_Lvl+"dict_origins_isolated.pk", 'wb') as f: pkl.dump(dict_origins_isolated, f)

In [ ]:
del grid_26, dict_origins_isolated; gc.collect()

---

At this point, we cleared the 26's from the pairs.

Next, we want to reduce the number of false pairs before a final comprehensive check, as to save computational time.

First, we check if a cell in a pair only neighbors one other cell that iself neighbors (and only it) a lower neighbor, thus not being recognized as part of a pair, leaving the previous cell isolated, but labeled as a pair.

Then, we give an idex number to each pair origin using the label() function and connect the indices from one side to the other (remember, we found pairs using the shift method, so even if a cell looks isolated on the edge of the cube, it was found as part of a pair if it should've been). Then we do what we must do to only give one index to all the connected pairs.

Final check! Make sure if we subtract from each pair origin (at its edges) any of its surrounding (outside) cells, the result is not zero, because it would mean the origin pair has an elements of the same density that cannot be part of an origin, thus the whole pair cannot be.

---

### Check which pair elements ended up isolated

First, we check if a cell in a pair only neighbors one other cell that iself neighbors (and only it) a lower neighbor, thus not being recognized as part of a pair, leaving the previous cell isolated, but labeled as a pair.

In [ ]:
empty_pairs = True
if np.sum(grid_pairs) != 0:
    empty_pairs = False
    
    grid_pairs_fake = grid_pairs.copy()
    
    for offs in offsets: grid_pairs_fake &= ~shift_cube(grid_pairs, offs)
    
    grid_pairs &= ~grid_pairs_fake

    if np.sum(grid_pairs) == 0: empty_pairs = True

In [ ]:
del grid_pairs_fake; gc.collect()

### Connect the pairs' indices from one side to the other

Make a list of indices for each pair using label() and match them from one-side of the grid to another.

Remember, we found pairs using the shift method, so even if a cell looks isolated on the edge of the cube, it was found as part of a pair if it should've been.

In [ ]:
empty_labels_pairs = True
if not empty_pairs:
    labeled, num_features = label(grid_pairs, structure=np.full((3, 3, 3), True))
    labels_pairs = labeled_pairs(labeled)

    if len(labels_pairs) != 0 :
        # remove duplicates regardless of order and maintain an increasing in index ordering
        labels_pairs = [list(t) for t in {tuple(sorted(i)) for i in labels_pairs}]
        empty_labels_pairs = False

### Merge [a,b] and [b,c] into [a,b,c]

In [ ]:
if not empty_labels_pairs:
    # [a,b]+[c,b]->[a,b,c]
    labels_pairs = combine_lists(labels_pairs)

    # [a,b,c]->[a]
    abc_a(np.array([len(i) for i in labels_pairs]), 
          np.array([coord for sublist in labels_pairs for coord in sublist]), 
          labeled)

### Check the pairs' neighbors

Final check! Make sure if we subtract from each pair origin (at its edges) any of its surrounding (outside) cells, the result is not zero, because it would mean the origin pair has an elements of the same density that cannot be part of an origin, thus the whole pair cannot be.

(We could have just had this check alone, but with the previous ones we saved a lot of computational time.)

In [ ]:
if not empty_pairs:
    grid_fft_max         = np.max(grid_lvld)
    grid_to_subtract     = grid_lvld.copy()
    grid_subtracted_from = grid_lvld # No need to copy, since we don't use grid_lvld after this
    
    grid_to_subtract[     grid_pairs] = -1
    grid_subtracted_from[~grid_pairs] = grid_fft_max+1

In [ ]:
del grid_pairs; gc.collect()

In [ ]:
if not empty_pairs:
    fake_indices = []
    
    for offs in offsets:
    
        # Get the grid of origin pairs equal to their surroundings.
        grid_diff = grid_subtracted_from - shift_cube(grid_to_subtract, offs)
    
        # Get from that grid all the cooresponding labeled values from its grid, take these values only
        #     once (using list(set())) in a list and save them as fake indices.
        for i in list(set(labeled[grid_diff == 0].tolist())): fake_indices.append(i)
    
    # set() them and remove them.
    fake_indices_into_labeled(np.array(list(set(fake_indices))), labeled)

In [ ]:
if not empty_pairs: del grid_diff, grid_to_subtract, grid_subtracted_from, fake_indices; gc.collect()

### Make the dict of pairs

In [ ]:
dict_origins_pairs = {val: [] for val in range(0, lvl)}

In [ ]:
if not empty_pairs:
    list_pairs = list_pairs_maker(labeled)

    with open(file_path_Lvl+"grid_lvld.pk", 'rb') as f: grid_lvld = pkl.load(f)

    for pairs_i in list_pairs: dict_origins_pairs[grid_lvld[tuple(pairs_i[0])]].append(pairs_i)

In [ ]:
with open(file_path_Lvl+"dict_origins_pairs.pk", 'wb') as f: pkl.dump(dict_origins_pairs, f)

---
---
---